In [ ]:
from VisionEngine.datasets import guppies
from VisionEngine.utils.config import process_config
from VisionEngine.utils import factory
from VisionEngine.utils.eval import embed_images
from VisionEngine.extensions import latent_evolution
from VisionEngin.plotting import visualize_generation

import os
from dotenv import load_dotenv
from pathlib import Path

import numpy as np
import tensorflow as tf

In [ ]:
# If you only have one GPU, this must = 0
GPU = 0

In [ ]:
env_path = Path('../') / '.env'
load_dotenv(dotenv_path=env_path)

**Setup the model and data loader**

In [ ]:
checkpoint_path = os.path.join(
    os.getenv("VISIONENGINE_HOME"),
    "checkpoints/guppies_DHRL_model.hdf5"
)
config_file = os.path.join(
    os.getenv("VISIONENGINE_HOME"),
    "VisionEngine/configs/guppies_DHRL_config.json"
)

In [ ]:
config = process_config(config_file)

In [ ]:
data_loader = factory.create(
            "VisionEngine.data_loaders."+config.data_loader.name
            )(config)

In [ ]:
with tf.device(f"GPU:{GPU}"):
    model = factory.create(
                "VisionEngine.models."+config.model.name
                )(config)
    model.load(checkpoint_path)
    model.trainable = False

In [ ]:
data_loader = factory.create(
            "VisionEngine.data_loaders."+config.data_loader.name
            )(config)

**Take a random sample as a starting population**

In [ ]:
with tf.device(f"GPU:{GPU}"):
    parents_orig = []
    for image, label in data_loader.get_test_data():
        parents_orig.append(image.numpy())
    parents_orig = np.vstack(parents_orig)
    parents = embed_images(data_loader.get_test_data(), model)
    parents = tf.transpose(tf.Variable(parents), (1,0,2))
    parents = parents[:900]

**Simulate generations**

*See: VisionEnging.extensions.evolve_population*

In [ ]:
with tf.device(f"GPU:{GPU}"):
    parent_record = evolve_population(parents, model, parents_orig, 
                                      temperature=0.2, persistence=0.5,
                                      mutation_per = 0.1, N_GENERATIONS = 500, 
                                      POPULATION_SIZE = 1000, BATCH_SIZE = 100):
    pr = np.stack(parent_record[1:]).reshape(500,1000, 40) 
    gens = TSNE(callbacks=ErrorLogger(), n_jobs=8).fit(pr)

**Visulaize generations and alleles**

In [ ]:
with tf.device(f"GPU:{GPU}"):
    generation = visualize_generation(generation, parent_record)

In [ ]:
plt.imshow(generation)